In [ ]:
import pandas as pd
import datetime
import time
from datetime import timezone
import pytz
import matplotlib.pyplot as plt
from google.colab import files
import random
!pip install imageio_ffmpeg
from moviepy.video.io.VideoFileClip import VideoFileClip
from math import ceil
import numpy as np
!pip install heartpy
import heartpy as hp
!pip install hrv-analysis
import hrvanalysis as hrv
from hrvanalysis import get_frequency_domain_features
!pip install biosppy
import numpy as np
from scipy.signal import butter, filtfilt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 9.0 MB/s eta 0:00:00


In [ ]:
for x in range(4,11):
  x=str(x)
  E4_data_path='/content/drive/MyDrive/Data/E4/S'+x+'_M/Study/BVP.csv' #BVP
  E4_HR_data_path ='/content/drive/MyDrive/Data/E4/S'+x+'_M/Study/HR.csv'
  E4_IBI_data_path ='/content/drive/MyDrive/Data/E4/S'+x+'_M/Study/IBI.csv'

  BVP = pd.read_csv(E4_data_path)
  HR = pd.read_csv(E4_HR_data_path)
  IBI = pd.read_csv(E4_IBI_data_path)
  BVP = BVP.iloc[1:]
  HR = HR.iloc[1:]
  BVP.columns = ['BVP']
  HR.columns = ['HR']


  # Load CSV file
  data = pd.read_csv(E4_data_path)
  freq_E4=data.iloc[0]
  #data.columns = data.columns.str.replace('.00', '')
  unix_time = data.columns.values.tolist()
  start = unix_time[0].replace('.00','')
  time = float(start)
  data = data.iloc[1:]
  data.index = range(len(data))
  count=1
  microseconds =0.015625

  for i in range(0,len(data)):
    if i>0:
      time +=microseconds
      unix_time.append(time)
    else:
      #print(time)
      unix_time.append(time)

  unix_time.pop(0)
  unix_df = pd.DataFrame(unix_time)
  unix_df.columns = ['Unix_timestamp']
  data.columns = ['BVP']

  E4_data = pd.merge(unix_df,data, left_index=True,right_index=True)

  unix_df.columns = ['Time']
  BVP_Time_Data = pd.merge(unix_df,data, left_index=True,right_index=True)
  BVP_Time_Data.loc[:, 'Time'] = BVP_Time_Data.loc[:, 'Time'].apply(lambda x: '{:.6f}'.format(round(x, 6)))

  Start_Time = IBI.columns.values.tolist()
  Start_Time = float(Start_Time[0])
  #print(Start_Time)
  IBI.columns.values[0] = 'Time'
  #IBI = IBI.rename(columns={'1677107700.000000': 'Time'})
  #IBI = IBI.rename(columns={' IBI': 'IBI'})
  IBI.columns.values[1] = "IBI"
  IBI.loc[:, 'Time'] = IBI.loc[:, 'Time'].apply(lambda x: float(x) + Start_Time)
  IBI.loc[:, 'Time'] = IBI.loc[:, 'Time'].apply(lambda x: '{:.6f}'.format(round(x, 6)))
  IBI.loc[:, 'IBI'] = IBI.loc[:, 'IBI'].apply(lambda x: '{:.6f}'.format(round(x, 6)))

    # Define bandpass filter parameters
  lowcut = 0.5  # lower cutoff frequency in Hz
  highcut = 10.0  # upper cutoff frequency in Hz
  fs = 64.0  # sampling frequency in Hz
  nyquist = 0.5 * fs  # Nyquist frequency

  # Compute filter coefficients
  b, a = butter(2, [lowcut/nyquist, highcut/nyquist], btype='band')

  # Define window size for moving average filter
  window_size = 5

  # Load BVP data from file or other source
  # Assumes BVP data is a 1D numpy array of BVP values over time
  bvp_data = BVP_Time_Data
  time_col = bvp_data["Time"].values
  bvp_col = bvp_data["BVP"].values


  # Apply bandpass filter to remove noise components
  filtered_bvp = filtfilt(b, a, bvp_col)

  # Square the filtered data to highlight peaks
  squared_bvp = np.square(filtered_bvp)

  # Apply a moving average filter to smooth close-by peaks
  #moving average filtering may distort the frequency content of the signal and should be avoided for frequency-domain HRV analysis.
  moving_avg_bvp = np.convolve(squared_bvp, np.ones(window_size)/window_size, mode='valid')

  # combine the new BVP values with time column
  filtered_bvp_time = pd.DataFrame({'Time': time_col, 'Filtered_bvp': filtered_bvp})
  squared_bvp_time = pd.DataFrame({'Time': time_col, 'Squared_bvp': squared_bvp})
  bvp_moving_avg_time = pd.DataFrame({'Time': time_col[window_size-1:], 'Moving_ave_bvp': moving_avg_bvp})

  import pandas as pd
  import numpy as np
  import matplotlib.pyplot as plt
  import scipy.signal as signal
  #import hrvanalysis as hrv
  #import biosppy.signals.bvp as bvp

  # Load BVP and IBI data
  bvp_data = filtered_bvp_time
  ibi_data = IBI

  initial_time = ibi_data['Time'].iloc[0]
  initial_time = float(initial_time)
  bvp_interval_list =[]

  def find_end_time(initial_time,start_row):
    #I'll be using 5 minutes to start since all studies mention at least 5 minutes to detect a good hrv signal(This ammount of time can change however I wan't) Just have to change n in 60*n below. Being n number of minutes
    initial_time = float(initial_time)+float(60*1.5)
    for i, row in  ibi_data.iloc[start_row:].iterrows():
      temp_time = float(row["Time"]) + float(row["IBI"])
      if temp_time >= initial_time:
        end_time = str(float(row["Time"]) + float(row["IBI"]))
        return end_time,i+1,False
      if(i+1 == len(ibi_data)):
        end_time = str(float(row["Time"]) + float(row["IBI"]))
        return end_time,start_row,True

  def find_init_time(start_row):
    start_row = int(start_row)
    return ibi_data['Time'].iloc[start_row]


  # Initialize lists to store HRV features
  rmssd_list = []
  sdnn_list = []
  pnn50_list = []
  vlf_list = []
  lf_list = []
  hf_list = []
  lf_hf_list = []
  time_list = []
  bvp_interval_list =[]

  #print(initial_time)
  end_time,start_row,end = find_end_time(initial_time,0)

  #print(initial_time)
  #print(end_time)
  #print(start_row)
  end_cycle=False
  while end_cycle==False:
    if(end==True):
      end_cycle=True

    bvp_interval = bvp_data.loc[(bvp_data["Time"] >= str(initial_time)) & (bvp_data["Time"] <= str(end_time))]

    if not bvp_interval.empty:
          # Calculate HRV features
          ibi_diff = np.diff(bvp_interval["Filtered_bvp"])

          # Skip interval if ibi_diff is empty
          if len(ibi_diff) > 0:
              #print(len(bvp_interval["Filtered_bvp"]))
              #print(len(ibi_diff))
              #print(f"Processing row {i}: start_time={start_time}, end_time={end_time}, IBI={row['IBI']}")
              rmssd = np.sqrt(np.mean(ibi_diff ** 2))
              sdnn = np.std(bvp_interval["Filtered_bvp"])
              pnn50 = sum(abs(ibi_diff) > 0.05) / len(ibi_diff)


              # Compute frequency-domain HRV features
              fs = 64 # Sampling frequency of BVP signal
              f, psd = signal.welch(bvp_interval["Filtered_bvp"], fs=fs, nperseg=5000)
              vlf = np.sum(psd[(f >= 0.003) & (f < 0.04)])
              lf = np.sum(psd[(f >= 0.04) & (f < 0.15)])
              hf = np.sum(psd[(f >= 0.15) & (f < 0.4)])
              lf_hf = lf / hf


              # Append HRV features to lists
              time_list.append(bvp_interval["Time"])
              bvp_interval_list.append(bvp_interval["Filtered_bvp"])
              rmssd_list.append(rmssd)
              sdnn_list.append(sdnn)
              pnn50_list.append(pnn50)
              vlf_list.append(vlf)
              lf_list.append(lf)
              hf_list.append(hf)
              lf_hf_list.append(lf_hf)


    initial_time = find_init_time(start_row)
    end_time,start_row,end = find_end_time(initial_time,start_row)
    #print(initial_time)
    #print(end_time)

  hrv_data = pd.DataFrame({"Time": time_list,
                          "BVP":  bvp_interval_list,
                          "RMSSD": rmssd_list,
                          "SDNN": sdnn_list,
                          "pNN50": pnn50_list,
                          "VLF": vlf_list,
                          "LF": lf_list,
                          "HF": hf_list,
                          "LF/HF": lf_hf_list})

  #print(hrv_data.head())
  hrv_data.to_csv('/content/drive/MyDrive/Data/E4/S'+x+'_M/Study/HRV1_5.csv', encoding = 'utf-8-sig')
  print(f'File {x} saved.')


  warnings.warn('nperseg = {0:d} is greater than input length '



File 4 saved.


  warnings.warn('nperseg = {0:d} is greater than input length '



File 5 saved.


  warnings.warn('nperseg = {0:d} is greater than input length '



...

File 8 saved.


  warnings.warn('nperseg = {0:d} is greater than input length '



File 9 saved.


  warnings.warn('nperseg = {0:d} is greater than input length '



File 10 saved.
